In [146]:
import sumolib 
import traci
from sumolib import checkBinary

In [147]:
junction = {'KlongTei': 'cluster_1892287670_272491964_272492178',
      'Rama4': 'cluster_272488163_282390730_66263210_66263222',
      'NaRanong': 'cluster_272488164_272492179_3457051443_61907354',
      'Sunthon': 'gneJ83',
      'KasemRat': 'cluster_272448137_272555800_272555808_7660045934_7710268409',
      'Virtual' : '270329335'}
direction = {'KlongTei' : ['NB','WB','EB'], 
             'Rama4': ['WB','SB','EB','NB'],
             'NaRanong': ['NB','WB','SW','SB','EB'],
             'Sunthon': ['WB','SB','EB','NB'],
             'KasemRat': ['WB','EB','NB'],
             'Virtual' : ['SB','EB']}

In [148]:
# KlongTei = 'cluster_1892287670_272491964_272492178'
# Rama4 = 'cluster_272488163_282390730_66263210_66263222'
# NaRanong = 'cluster_272488164_272492179_3457051443_61907354'
# Sunthon = 'gneJ83'
# KasemRat = 'cluster_272448137_272555800_272555808_7660045934_7710268409'
# Virtual = '270329335'
# junction = [KlongTei,Rama4,NaRanong,Sunthon,KasemRat,Virtual]

In [149]:
def reset():
    sumoBinary = checkBinary('sumo-gui')
    traci.start([sumoBinary, "-c", "Kasemrat-SUMO-UsingBookNetFile/osm.sumocfg",
                             "--tripinfo-output", "tripinfo.xml", '--start','true','--quit-on-end','true','--time-to-teleport','-1'])

In [150]:
net1 = sumolib.net.readNet('Kasemrat-SUMO-UsingBookNetFile/AddLane.net.xml')

In [195]:
def get_connect(ne):
    dir_con = set()
    out = ne.getToNode().getOutgoing()
    for o in out:
#                                 print(o)
        con = sumolib.net.edge.Edge.getConnections(ne,o)
#                                 print(con)
        for c in con:
            dir_con.add(sumolib.net.connection.Connection.getDirection(c).upper())
    connect = ''
    if 'L' in dir_con : connect += 'L'
    else: connect += 'X'
    if 'S' in dir_con : connect += 'S'
    else: connect += 'X'
    if 'R' in dir_con : connect += 'R'
    else: connect += 'X'
    if 'U' in dir_con : connect += 'U'
    else: connect += 'X'
    return connect

In [153]:
reset()

In [152]:
traci.close()

In [203]:
lane_name = {}
last_edge = None
NoNeed = ['27702347#6', '300813736#0', '825786399','459492913#3', '459492913#2','459492913#0','618739073#1',
          '825786398', '482384842#2']
for j in junction.keys(): 
    if j == 'Sunthon':
        dir_index = -1
        EdgeIndex = 1
        lane_1 = list(traci.trafficlight.getControlledLanes(junction[j]))
        for l_1 in lane_1:
            if l_1[-1] == '0' and l_1[:-2] != last_edge:
                EdgeIndex = 1
                dir_index += 1
                edge = l_1[:-2]
                for i in range(10):
                    nextEdges = net1.getEdge(edge).getFromNode().getIncoming()
                    for ne in nextEdges:
                        connect = get_connect(ne)
#                         print(connect)
                        EdgeIndex += 1
                        edgeID = ne
                        numOfLane = edgeID.getLaneNumber()
                        for LaneIndex in range(numOfLane):
                            nextLanes = edgeID.getID()+'_'+str(LaneIndex)
                            if 'R' in connect and LaneIndex == numOfLane-1:
                                lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(EdgeIndex)+'_'+str(LaneIndex)+'_'+connect] = nextLanes
                            elif 'L' in connect and LaneIndex == 0:
                                lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(EdgeIndex)+'_'+str(LaneIndex)+'_'+connect] = nextLanes

                            else:
                                lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(EdgeIndex)+'_'+str(LaneIndex)+'_'+'XSXX'] = nextLanes
                        edge = edgeID.getID()
            connect = get_connect(net1.getEdge(l_1[:-2]))
            if 'R' in connect and LaneIndex == numOfLane-1 and 'U' not in connect:
                lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(1)+'_'+str(LaneIndex)+'_'+'XSRX'] = nextLanes
            elif 'R' in connect and LaneIndex == numOfLane-1 and 'U' in connect:
                lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(1)+'_'+str(LaneIndex)+'_'+'XSRU'] = nextLanes
            elif 'L' in connect and LaneIndex == 0:
                lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(1)+'_'+str(LaneIndex)+'_'+'LSXX'] = nextLanes
            else:
                lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(1)+'_'+str(LaneIndex)+'_'+'XSXX'] = nextLanes
            last_edge = l_1[:-2]
for l in sorted(lane_name):
# for l in lane_name:
    print(l, '=', "'"+lane_name[l]+"'")

SUNTHON_EB_10_0_XSXX = '824116551#0_0'
SUNTHON_EB_10_1_XSRX = '824116551#0_1'
SUNTHON_EB_11_0_LSXX = '157063979_0'
SUNTHON_EB_11_1_XSXX = '157063979_1'
SUNTHON_EB_11_2_XSXX = '157063979_2'
SUNTHON_EB_12_0_XSXX = '-552779154#4_0'
SUNTHON_EB_12_1_XSXX = '-552779154#4_1'
SUNTHON_EB_13_0_XSXX = '157063985#0_0'
SUNTHON_EB_13_1_XSXX = '157063985#0_1'
SUNTHON_EB_14_0_LSRX = '25053626_0'
SUNTHON_EB_14_1_LSRX = '25053626_1'
SUNTHON_EB_15_0_LXRX = '338595767_0'
SUNTHON_EB_15_1_XSXX = '338595767_1'
SUNTHON_EB_15_2_XSXX = '338595767_2'
SUNTHON_EB_15_3_LXRX = '338595767_3'
SUNTHON_EB_16_0_LSRX = '824116544_0'
SUNTHON_EB_16_1_XSXX = '824116544_1'
SUNTHON_EB_16_2_LSRX = '824116544_2'
SUNTHON_EB_17_0_LSRX = '824116550#0_0'
SUNTHON_EB_17_1_LSRX = '824116550#0_1'
SUNTHON_EB_18_0_XSXX = '824116551#0_0'
SUNTHON_EB_18_1_XSRX = '824116551#0_1'
SUNTHON_EB_19_0_LSXX = '157063979_0'
SUNTHON_EB_19_1_XSXX = '157063979_1'
SUNTHON_EB_19_2_XSXX = '157063979_2'
SUNTHON_EB_1_1_XSRX = '157063985#0_1'
SUNTHON_EB_20_0_X